In [ ]:
import pandas as pd
import os
import altair as alt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox

df_clean = pd.read_csv("./data/Processed/employment_rate.csv")
df_clean['Month'] = pd.to_datetime(df_clean['Month'], errors='coerce')

industries_to_plot = [
    'Manufacturing',
    'Construction',
    'Retail trade',
    'Transportation and warehousing',
    'Leisure and hospitality',
    'Financial activities'
]

df_long = df_clean.melt(
    id_vars=['Month'],
    value_vars=industries_to_plot,
    var_name='Industry',
    value_name='Employment'
)

start_picker = widgets.SelectionSlider(
    options=list(df_clean['Month'].dt.to_period('M').astype(str)),
    description='Start Month',
    layout=widgets.Layout(width='800px')
)

end_picker = widgets.SelectionSlider(
    options=list(df_clean['Month'].dt.to_period('M').astype(str)),
    description='End Month',
    layout=widgets.Layout(width='800px')
)

btn = widgets.Button(description="Calculate % Change", button_style='success')
out = widgets.Output()

trend_chart = alt.Chart(df_long).mark_line().encode(
    x=alt.X('Month:T', title='Year'),
    y=alt.Y('Employment:Q', title='Employment (Thousands)'),
    color=alt.Color('Industry:N', scale=alt.Scale(scheme='category10')),
    tooltip=['Month:T', 'Industry:N', 'Employment:Q']
).properties(
    width=800,
    height=400,
    title='Employment Trend by Industry (2005–2025)'
)

display(trend_chart)

display(VBox([start_picker, end_picker, btn]))

display(out)

def on_button_clicked(b):
    with out:
        out.clear_output()

        start = pd.to_datetime(start_picker.value)
        end = pd.to_datetime(end_picker.value)

        if start >= end:
            print("⚠️ Start date must be earlier than End date!")
            return

        start_vals = df_long[df_long['Month'] == start]
        end_vals = df_long[df_long['Month'] == end]

        if start_vals.empty or end_vals.empty:
            print("⚠️ Selected dates not available in data!")
            return

        compare = pd.merge(
            start_vals,
            end_vals,
            on='Industry',
            suffixes=('_start', '_end')
        )

        compare['Change'] = (compare['Employment_end'] - compare['Employment_start']) / compare['Employment_end'] * 100

        bar = alt.Chart(compare).mark_bar().encode(
            x=alt.X('Industry:N', title='Industry'),
            y=alt.Y('Change:Q', title='Percentage Change'),
            color=alt.condition(
                alt.datum.Change > 0,
                alt.value('steelblue'),
                alt.value('orangered')
            ),
            tooltip=[
                alt.Tooltip('Industry:N'),
                alt.Tooltip('Change:Q', format='.2f')
            ]
        ).properties(
            width=800,
            height=300,
            title=f'Percentage Change from {start_picker.value} to {end_picker.value}'
        )

        display(bar)

btn.on_click(on_button_clicked)
